<a href="https://colab.research.google.com/github/danielbauer1860/LDS_Project/blob/main/generation/generating_news_4epoch_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This notebook references [the project of Salminen et al. (2021)](https://github.com/joolsa/FakeReviews/blob/)

In [2]:
!pip install iteround -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading the model

Redownloading the model required some tweaks, which are described in:
https://stackoverflow.com/questions/76459034/how-to-load-a-fine-tuned-peft-lora-model-based-on-llama-with-huggingface-transfo

In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import infer_auto_device_map, init_empty_weights

peft_model_id = 'dbauer1860/llama-2-bnc-baby-news-4-epochs'

config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype='auto',
    device_map='auto',
    offload_folder="offload", offload_state_dict = True
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

# Extracting Word Counts and Prompts

In [5]:
import pandas as pd
import numpy as np
import random
import nltk
from iteround import saferound
from nltk.tokenize import word_tokenize

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
df = pd.read_csv('/content/drive/MyDrive/Linguistic Data Science/data/bnc_baby_texts.csv', sep='|')

In [8]:
df.head()

,text,category
0,"Why do certain images matter to one, and why i...",ACA
1,"In many developing countries, the immunogenici...",ACA
2,Anthony Kenny The British Library 1992 Scholar...,ACA
3,Beforehand he worked as a probation officer fo...,ACA
4,CONTINUITY AND CHANGE are features of every po...,ACA


In [9]:
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

In [10]:
df['sentences'] = df['text'].apply(split_into_sentences)

In [11]:
df

,text,category,sentences
0,"Why do certain images matter to one, and why i...",ACA,"[Why do certain images matter to one, and why ..."
1,"In many developing countries, the immunogenici...",ACA,"[In many developing countries, the immunogenic..."
2,Anthony Kenny The British Library 1992 Scholar...,ACA,[Anthony Kenny The British Library 1992 Schola...
3,Beforehand he worked as a probation officer fo...,ACA,[Beforehand he worked as a probation officer f...
4,CONTINUITY AND CHANGE are features of every po...,ACA,[CONTINUITY AND CHANGE are features of every p...
...,...,...,...
177,"Artworks THE Friends of Oriel Ynys Mon, Llange...",NEWS,"[Artworks THE Friends of Oriel Ynys Mon, Llang..."
178,THERE will be those who portray the report of ...,NEWS,[THERE will be those who portray the report of...
179,BUCKINGHAM PALACE 30 September: The Princess R...,NEWS,[BUCKINGHAM PALACE 30 September: The Princess ...
180,The Scottish Landowners' Federation unequivoca...,NEWS,[The Scottish Landowners' Federation unequivoc...


In [12]:
random_state = 34
random_sentence_state = 1860
sample_count = 20

In [13]:
def get_random_sentence(sentences_list, random_state=random_sentence_state, min_word_count=5):
    random.seed(random_state)
    filtered_sentences = [sentence for sentence in sentences_list if len(word_tokenize(sentence)) > min_word_count]
    return random.choice(filtered_sentences) if filtered_sentences else None

In [14]:
df['prompt'] = df['sentences'].apply(get_random_sentence)

In [15]:
def get_samples(df, n=sample_count, state=random_state):
  return df.sample(n=n, random_state=state)

In [16]:
df_aca, df_dem, df_fic, df_news = [get_samples(y) for x, y in df.groupby(['category'])]

<ipython-input-16-c993aa370679>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  df_aca, df_dem, df_fic, df_news = [get_samples(y) for x, y in df.groupby(['category'])]


In [17]:
df_news

,text,category,sentences,prompt
100,HELP the Aged's shops are warming up for winte...,NEWS,[HELP the Aged's shops are warming up for wint...,Mrs Cross flew back to Manchester yesterday to...
109,"IF THIS WAS HAPPINESS by Barbara Leaming, Weid...",NEWS,"[IF THIS WAS HAPPINESS by Barbara Leaming, Wei...","Realising that she had not loved him, Boris th..."
95,This week's figures suggest that one in three ...,NEWS,[This week's figures suggest that one in three...,TRIPS to Ireland recently have reinforced my v...
181,An under-16s ladies soccer meeting will be hel...,NEWS,[An under-16s ladies soccer meeting will be he...,"Llanidloes finished on 127–9, in which Sean So..."
113,ELAINE HEATH likes pink eye glitter and a vodk...,NEWS,[ELAINE HEATH likes pink eye glitter and a vod...,But it is essentially up to the patient.’ He s...
126,"GEOFF Mulcahy, the soft-spoken chief executive...",NEWS,"[GEOFF Mulcahy, the soft-spoken chief executiv...",Hammerson shot up 45p to 855p in sympathy.
89,WILD BOAR once roamed the fields and forests o...,NEWS,[WILD BOAR once roamed the fields and forests ...,As Fazzi Brothers is to Glasgow and the west o...
177,"Artworks THE Friends of Oriel Ynys Mon, Llange...",NEWS,"[Artworks THE Friends of Oriel Ynys Mon, Llang...","Nasty man takes revenge, but stabs Lisa by mis..."
175,"MORE than 5,000 workers have swamped Hamilton ...",NEWS,"[MORE than 5,000 workers have swamped Hamilton...",The discounts include 3pc off the standard loa...
146,PERHAPS the Independent Traveller ought to car...,NEWS,[PERHAPS the Independent Traveller ought to ca...,Eating Out: Plenty of places in all price rang...


In [18]:
aca_prompts = df_aca['prompt'].to_list()
fic_prompts = df_fic['prompt'].to_list()
news_prompts = df_news['prompt'].to_list()

# Generating output

In [19]:
#Generation parameters; these were, for the most part, set in accordance to Salminen et al.
#the only difference is the implementation of a deviation value
do_sample = True
top_k = 0
top_p = 0.92
temperature = 0.7
eos_token = model.config.eos_token_id
device = "cuda:0"
max_length = 1000
repetition_penalty = 1.1

In [20]:
#from Salminen et al.; a seed value was set to allow the generation to be re-produced and continued over multiple sessions
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

The generating loop

In [21]:
def generate(prompts):
  outputs = []
  for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    random_seed(seed_value=random_state, use_cuda=True)
    o = model.generate(**inputs, do_sample=do_sample, top_k=top_k, top_p=top_p, temperature=temperature, eos_token_id=eos_token, max_length=max_length, repetition_penalty=repetition_penalty)
    outputs.append(tokenizer.decode(o[0], skip_special_tokens=True))
  return outputs

In [22]:
aca_outputs = generate(aca_prompts)

In [23]:
fic_outputs = generate(fic_prompts)

In [24]:
news_outputs = generate(news_prompts)

In [25]:
aca_results = pd.DataFrame(aca_outputs, columns=['output'])
aca_results['prompt_category'] = 'ACA'
aca_results['prompt'] = aca_prompts

fic_results = pd.DataFrame(fic_outputs, columns=['output'])
fic_results['prompt_category'] = 'FIC'
fic_results['prompt'] = fic_prompts

news_results = pd.DataFrame(news_outputs, columns=['output'])
news_results['prompt_category'] = 'NEWS'
news_results['prompt'] = news_prompts

result_frames = [aca_results, fic_results, news_results]

result_df = pd.concat(result_frames, ignore_index=True)
result_df

,output,prompt_category,prompt
0,The head then commented on his social developm...,ACA,The head then commented on his social developm...
1,Conclusions: Policy Rhetoric and Structural Re...,ACA,Conclusions: Policy Rhetoric and Structural Re...
2,One advantage is the ability to model a range ...,ACA,One advantage is the ability to model a range ...
3,"These are culpable acts, often regarded as bei...",ACA,"These are culpable acts, often regarded as bei..."
4,Genesis: An Authorship Study in Computer-assis...,ACA,Genesis: An Authorship Study in Computer-assis...
5,One idea about the life habits of the dinosaur...,ACA,One idea about the life habits of the dinosaur...
6,It is not in those circumstances surprising th...,ACA,It is not in those circumstances surprising th...
7,In response to the e.m.f. (electromotive force...,ACA,In response to the e.m.f.
8,The initial deformation corresponds to portion...,ACA,The initial deformation corresponds to portion...
9,We illustrate reduction to upper Hessenberg fo...,ACA,We illustrate reduction to upper Hessenberg fo...


In [26]:
result_df.to_csv('/content/drive/MyDrive/Linguistic Data Science/data/4epoch-news-output.csv', index=False)